## XGBoost
This notebook supports the DengAI project and is for evaluating KGBoost Models. For the supporting utilities, see the DengAI-Utils Notebook

In [4]:
#Run the utils notebook
%run DengAI-Utils.ipynb

Environment is: base


In [5]:
#pre-process the data
df_x_train_sj, df_y_train_sj, df_x_test_sj, df_y_test_sj, df_x_valid_sj, df_y_valid_sj, df_x_pred_sj, df_y_pred_sj=pre_process_data(
    city='sj',
    train_split=0.8,
    test_split=0.2,
    valid_split=0.0,
    xy_split=0.0,
    time_series_split=True
)
#set target field
target='total_cases'

In [6]:
#Pre-process data for iq
df_x_train_iq, df_y_train_iq, df_x_test_iq, df_y_test_iq, df_x_valid_iq, df_y_valid_iq, df_x_pred_iq, df_y_pred_iq=pre_process_data(
    city='iq',
    train_split=0.8,
    test_split=0.2,
    valid_split=0.0,
    xy_split=0.0,
    time_series_split=True
)

#set target field
target='total_cases'

In [7]:
#Run the model
'''Use XGBoost with a grid search to find hp's for each model'''

#Start timer
from datetime import datetime
s_time=datetime.now()

#Define param grid
p_grid={
    'n_estimators':[10,100,200,300],
    'reg_lambda':[0,1,5,10],
    'reg_alpha':[0,1,5,10],
    'gamma':[0,1,10],
    'max_depth':[3,6,10,20,25],
    'learning_rate':[0.0000001,0.00001,0.0001,0.001,0.01]
}

#run param grid for sj
grid_sj=GridSearchCV(XGBRegressor(),p_grid,cv=5,return_train_score=True)
grid_sj.fit(df_x_train_sj,df_y_train_sj[target])

#run param grid for iq
grid_iq=GridSearchCV(XGBRegressor(),p_grid,cv=5,return_train_score=True)
grid_iq.fit(df_x_train_iq,df_y_train_iq[target])

#Evaluate results
print('Score of SJ train: ' + str(grid_sj.score(df_x_train_sj,df_y_train_sj[target])))
print('Score of SJ test: ' + str(grid_sj.score(df_x_test_sj,df_y_test_sj[target])))
print('Score of IQ train: ' + str(grid_iq.score(df_x_train_iq,df_y_train_iq[target])))
print('Score of IQ test: ' + str(grid_iq.score(df_x_test_iq,df_y_test_iq[target])))
print('-'*25)
evaluate_results(grid_sj,grid_iq,df_x_test_sj,df_x_test_iq,df_y_test_sj,df_y_test_iq,target='total_cases')
print('-'*25)
#end timer
e_time=datetime.now()
exec_time=e_time-s_time
print('Execution time:' + str(exec_time))


Score of SJ train: 0.1199582458311278
Score of SJ test: -0.29555687906009065
Score of IQ train: 0.6945428994337732
Score of IQ test: 0.6216476072718999
-------------------------
MAE of SJ: 20.17594733954854
MAE of IQ: 4.89117611249288
MAE of Combined: 14.945417261848885
-------------------------
Execution time:10:59:21.068778


In [8]:
'''Create the submission file'''
#Use the trained model to make predictions on the holdout set
y_pred_sj=grid_sj.predict(df_x_pred_sj)
y_pred_iq=grid_iq.predict(df_x_pred_iq)

#Create the holdout file
create_submit_file(y_pred_sj,y_pred_iq)

#### Results
Using the XGBoost and combining with features engineered by through means and aggregations over rolling time periods and then lagging those features, the best models were used from the grid search. Results were slightly better than the baseline model